In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
import torch_geometric
print(torch_geometric.__version__)

In [2]:
# data_loader.py

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric.utils import dense_to_sparse

class SubjectDataset(Dataset):
    def __init__(self, high_dim_features, low_dim_features, labels):#,edge_index):
        # 由于我们将在模型中处理低维特征的嵌入，所以我们在这里不需要转换为Tensor
        self.high_dim_features = high_dim_features
        self.low_dim_features = low_dim_features
        self.labels = labels
        # self.edge_index = edge_index

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        high_dim_sample = torch.tensor(self.high_dim_features.iloc[idx].values, dtype=torch.float)
        low_dim_sample = self.low_dim_features.iloc[idx].values  # 作为NumPy数组保持
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return high_dim_sample, low_dim_sample, label #,self.edge_index

def create_full_connected_edge_index(num_nodes):
    # 生成一个全连接网络的邻接矩阵
    adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
    # 将邻接矩阵转换为边索引
    edge_index, _ = dense_to_sparse(adj_matrix)
    return edge_index

def load_and_align_data(high_dim_path, low_dim_path, labels_path, test_size=0.2, val_size = 0.1,random_state=42):
    # 加载数据
    high_dim_df = pd.read_csv(high_dim_path)
    low_dim_df = pd.read_csv(low_dim_path)
    labels_df = pd.read_csv(labels_path)

    # 假设第一列是subject_id，对齐数据
    merged_df = high_dim_df.merge(low_dim_df, on='PTID',how = 'inner').merge(labels_df, on='PTID',how = 'inner')

    # 假设'high_dim_data'和'low_dim_data'分别是包含在您的CSV文件列名中的高维和低维数据标识
    high_dim_cols = [col for col in merged_df.columns if 'high_' in col]
    low_dim_cols = [col for col in merged_df.columns if 'low_' in col]
    labels = merged_df['label_DX'].values

    # 分离出高维和低维特征
    high_dim_features = merged_df[high_dim_cols]
    low_dim_features = merged_df[low_dim_cols]

    # 划分训练集和测试集
    initial_train_high, test_high, initial_train_low, test_low, initial_train_labels, test_labels = train_test_split(
        high_dim_features, low_dim_features, labels, test_size=test_size, random_state=random_state, stratify=labels)
    
     # 进一步划分出验证集
    train_high, val_high, train_low, val_low, train_labels, val_labels = train_test_split(
        initial_train_high, initial_train_low, initial_train_labels, test_size=val_size, random_state=random_state, stratify=initial_train_labels)

    # 在这里调用 create_full_connected_edge_index 函数来创建 edge_index
    # num_nodes = len(merged_df)
    # edge_index = create_full_connected_edge_index(num_nodes)

    # 创建数据集实例
    train_dataset = SubjectDataset(train_high, train_low, train_labels)#, edge_index)
    val_dataset = SubjectDataset(val_high, val_low, val_labels)#, edge_index)
    test_dataset = SubjectDataset(test_high, test_low, test_labels)#, edge_index)

    return train_dataset, val_dataset, test_dataset

def create_data_loader(dataset, batch_size=32, shuffle=True):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader


In [4]:
os.getcwd()


'/mnt/AD_Classification'

In [5]:
import os
import pandas as pd

os.getcwd()
os.listdir()
#os.getcwd()

df = pd.read_csv("data/FINAL_MRI_DATA1.csv")

df

def create_full_connected_edge_index(num_nodes):
    # 生成一个全连接网络的邻接矩阵
    adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
    # 将邻接矩阵转换为边索引
    edge_index, _ = dense_to_sparse(adj_matrix)
    return edge_index

create_full_connected_edge_index(1010).size()

torch.Size([2, 1019090])

In [9]:

# 示例用法
high_dim_path = 'data/FINAL_MRI_DATA1.csv'
low_dim_path = 'data/FINAL_TABLEDATA_MRI.csv'
labels_path = 'data/Y_label_NEW_01.csv'


train_dataset, test_dataset ,validation_dataset= load_and_align_data(high_dim_path, low_dim_path, labels_path)
train_loader = create_data_loader(train_dataset)
test_loader = create_data_loader(test_dataset, shuffle=False)
validation_loader = create_data_loader(validation_dataset, batch_size=2)

print(len(train_dataset))
print(len(test_dataset))

# 测试一些样本
for i in range(3):
    high, low, label = train_dataset[i]
    print(f"Sample {i} - High dim features: {high.shape}, Low dim features: {low.shape}, Label: {label}")
    

for batch_idx, (high, low, label) in enumerate(validation_loader):
    print(f"Batch {batch_idx} - High dim batch shape: {high.shape}, Low dim batch shape: {len(low)}, Labels in batch: {label}")
    if batch_idx == 2:  # 只测试前几个批次
        break


727
81
Sample 0 - High dim features: torch.Size([498]), Low dim features: (17,), Label: 0
Sample 1 - High dim features: torch.Size([498]), Low dim features: (17,), Label: 1
Sample 2 - High dim features: torch.Size([498]), Low dim features: (17,), Label: 0
Batch 0 - High dim batch shape: torch.Size([2, 498]), Low dim batch shape: 2, Labels in batch: tensor([0, 0])
Batch 1 - High dim batch shape: torch.Size([2, 498]), Low dim batch shape: 2, Labels in batch: tensor([0, 1])
Batch 2 - High dim batch shape: torch.Size([2, 498]), Low dim batch shape: 2, Labels in batch: tensor([1, 0])


In [91]:
os.getcwd()

'/mnt/AD_Classification/data'

In [29]:
# config.py

config = {
  "data": {
    "high_dim_path": 'data/FINAL_MRI_DATA1.csv',
    "low_dim_path": 'data/FINAL_TABLEDATA_MRI.csv',
    "labels_path": 'data/Y_label_NEW_01.csv',
    "batch_size": 128,
    "shuffle": True,
    "test_size":0.1,
    "val_size":0.1,
    "random_state":12345
  },
  "model": {
    "type": "GCN",
    "high_dim_input_size": 498,  
    "low_dim_input_size":17,
    "embedding_dim":56,
    "output_dim":2,
    "hidden_channels":128,
    "num_heads":4 
  },
  "train": {
    "epochs": 250,
    "learning_rate": 0.01,
    "device": "cuda:1"
  },
  "earlystopping":{
    "patience":5,
    "delta":0.01
  }
}

In [31]:
# train.py

import torch
import torch.nn as nn
import torch.optim as optim
import json
from data.data_loader import load_and_align_data, create_data_loader
from models.model import CombinedGAT,CombinedGCN
from utils import EarlyStopping  # 假设你有评估和早停的辅助函数
from torch_geometric.data import Data
import logging
import torch.optim.lr_scheduler as lr_scheduler
from torch_geometric.utils import dense_to_sparse


# 加载配置文件
#with open('models/config.json', 'r') as config_file:
#    config = json.load(config_file)

# 设置训练设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 添加日志记录功能
logging.basicConfig(filename='training.log', level=logging.INFO, 
                    format='%(asctime)s:%(levelname)s:%(message)s')

# 加载数据并创建数据集
train_dataset, val_dataset, _ = load_and_align_data(high_dim_path = config['data']['high_dim_path'], 
                                                  low_dim_path = config['data']['low_dim_path'],
                                                  labels_path = config['data']['labels_path'],
                                                  test_size = config['data']['test_size'],
                                                  val_size = config['data']['val_size'],
                                                  random_state = config['data']['random_state'])

# 创建数据加载器
train_loader = create_data_loader(train_dataset, batch_size=config['data']['batch_size'], shuffle=config['data']['shuffle'])
val_loader = create_data_loader(val_dataset, batch_size=config['data']['batch_size'], shuffle=config['data']['shuffle'])

model = CombinedGAT(high_dim_input_size=config["model"]["high_dim_input_size"],  # 适当调整这些参数，这里可以写成config
                 low_dim_input_size=config["model"]["low_dim_input_size"],
                 embedding_dim=config["model"]["embedding_dim"],
                 output_dim=config["model"]["output_dim"],  # 根据您的任务调整
                 hidden_channels=config["model"]["hidden_channels"],
                 num_heads=config["model"]["num_heads"]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['train']['learning_rate'])
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5)

# 初始化早停对象
early_stopping = EarlyStopping(patience=config["earlystopping"]["patience"], delta=config["earlystopping"]["delta"])

best_val_loss = float('inf')

for epoch in range(config['train']['epochs']):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for high_dim_features, low_dim_features, labels in train_loader: # , edge_index in train_loader:
        
        batch_size = high_dim_features.size(0)  # 获取当前批次的大小
        
        # 为当前批次生成全连接的邻接矩阵
        adj_matrix = torch.ones((batch_size, batch_size)) - torch.eye(batch_size)
        edge_index, _ = dense_to_sparse(adj_matrix)
        
        
        # 准备数据
        high_dim_features = high_dim_features.to(device)
        low_dim_features = low_dim_features.to(device)
        labels = labels.to(device)
        edge_index = edge_index.to(device)

        # 前向传播
        outputs = model(high_dim_features, low_dim_features, edge_index)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = total_loss / len(train_loader)
    train_acc = 100 * correct / total

    # 计算验证集上的损失
    model.eval()
    val_total_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for high_dim_features, low_dim_features, labels in val_loader: # ,edge_index in val_loader:  # 假设你有一个验证集加载器val_loader
            
            batch_size = high_dim_features.size(0)  # 获取当前批次的大小
        
            # 为当前批次生成全连接的邻接矩阵
            adj_matrix = torch.ones((batch_size, batch_size)) - torch.eye(batch_size)
            edge_index, _ = dense_to_sparse(adj_matrix)
                
            high_dim_features = high_dim_features.to(device)
            low_dim_features = low_dim_features.to(device)
            labels = labels.to(device)
            edge_index = edge_index.to(device)

            outputs = model(high_dim_features, low_dim_features, edge_index)
            loss = criterion(outputs, labels)
            val_total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
    val_loss =  val_total_loss/len(val_loader)
    val_acc = 100 * val_correct / val_total
    scheduler.step(val_loss)
    
    logging.info(f'Epoch {epoch+1}, Train Loss: {train_loss}, Train Acc: {train_acc}, Val Loss: {val_loss}, Val Acc: {val_acc}')
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Train Acc: {train_acc}, Val Loss: {val_loss}, Val Acc: {val_acc}')
    
    # 早停检查和保存最佳模型
#    early_stopping(val_loss,model)
#    if early_stopping.early_stop:
#        print("Early stopping")
#        break


Epoch 1, Train Loss: 14870.695577366012, Train Acc: 50.488997555012226, Val Loss: 1022416.5, Val Acc: 36.26373626373626
Epoch 2, Train Loss: 47223.46878794261, Train Acc: 52.68948655256724, Val Loss: 1792125.5, Val Acc: 36.26373626373626
Epoch 3, Train Loss: 1892.0467991062574, Train Acc: 42.665036674816626, Val Loss: 2065283.375, Val Acc: 36.26373626373626
Epoch 4, Train Loss: 24170.352928715092, Train Acc: 63.81418092909536, Val Loss: 2407888.0, Val Acc: 36.26373626373626
Epoch 5, Train Loss: 50941.06322274038, Train Acc: 63.93643031784841, Val Loss: 2632141.25, Val Acc: 36.26373626373626
Epoch 6, Train Loss: 14061.340278523308, Train Acc: 61.36919315403423, Val Loss: 2716091.0, Val Acc: 36.26373626373626
Epoch 7, Train Loss: 0.6712416069848197, Train Acc: 57.21271393643032, Val Loss: 2579112.25, Val Acc: 36.26373626373626
Epoch 8, Train Loss: 0.6531868066106524, Train Acc: 63.93643031784841, Val Loss: 2717069.25, Val Acc: 36.26373626373626
Epoch 9, Train Loss: 0.6642505867140633, Tr